In [2]:
def extract_introduction_text(content_json):
    """
    Safely extract the '1 INTRODUCTION' section from a paper content.json.
    """
    # Try 'metadata' block first, fallback to root
    sections = []

    if "metadata" in content_json and isinstance(content_json["metadata"], dict):
        sections = content_json["metadata"].get("sections", [])
    else:
        sections = content_json.get("sections", [])

    for section in sections:
        heading = section.get("heading")
        if heading:
            heading_clean = heading.strip().upper()
            if heading_clean.startswith("1 INTRODUCTION"):
                return section.get("text", "")

    return ""  # return empty string if not found


In [4]:
import os
import json
import numpy as np
import faiss

base_dir = "../data/dataset"
conferences = [
    "ICLR_2017", "ICLR_2018", "ICLR_2019", "ICLR_2020",
    "NIPS_2016", "NIPS_2017", "NIPS_2018", "NIPS_2019"
]

titles = []
paper_ids = []
title_intro_texts = []
paper_decisions = {}
review_scores = {}

for conf in conferences:
    paper_dir = os.path.join(base_dir, f"{conf}/{conf}_paper")
    content_dir = os.path.join(base_dir, f"{conf}/{conf}_content")

    for fname in os.listdir(paper_dir):
        if fname.endswith(".json"):
            with open(os.path.join(paper_dir, fname), "r", encoding="utf-8") as f:
                paper = json.load(f)
                pid = paper.get("id")
                title = paper.get("title", "")
                decision = paper.get("decision", "Unknown")

                # Load introduction section
                intro_text = ""
                content_path = os.path.join(content_dir, f"{pid}.json")
                if os.path.exists(content_path):
                    with open(content_path, "r", encoding="utf-8") as cfile:
                        content = json.load(cfile)
                        intro_text = extract_introduction_text(content)

                if pid and title:
                    titles.append(title)
                    paper_ids.append(pid)
                    paper_decisions[pid] = decision
                    title_intro_texts.append(f"Title: {title}\n\nIntroduction: {intro_text}")


In [30]:
from sentence_transformers import SentenceTransformer
# Load Sentence-BERT model
model1 = SentenceTransformer("all-MiniLM-L6-v2")
model2 = SentenceTransformer("all-mpnet-base-v2")
model3 = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")



In [31]:
# Generate new embeddings
title_intro_embeddings_np1 = np.array(model1.encode(title_intro_texts, convert_to_tensor=False), dtype='float32')
title_intro_embeddings_np2 = np.array(model2.encode(title_intro_texts, convert_to_tensor=False), dtype='float32')
title_intro_embeddings_np3 = np.array(model3.encode(title_intro_texts, convert_to_tensor=False), dtype='float32')

# Create new FAISS index
index1 = faiss.IndexFlatL2(title_intro_embeddings_np1.shape[1])
index1.add(title_intro_embeddings_np1)

index2 = faiss.IndexFlatL2(title_intro_embeddings_np2.shape[1])
index2.add(title_intro_embeddings_np2)

index3 = faiss.IndexFlatL2(title_intro_embeddings_np3.shape[1])
index3.add(title_intro_embeddings_np3)

models =[model1,model2,model3]
indexes = [index1,index2,index3]


In [7]:
review_scores = {}

for conf in conferences:
    review_dir = os.path.join(base_dir, f"{conf}/{conf}_review")

    for fname in os.listdir(review_dir):
        if fname.endswith(".json"):
            with open(os.path.join(review_dir, fname), "r", encoding="utf-8") as f:
                data = json.load(f)
                pid = data.get("id")
                reviews = data.get("reviews", [])

                ratings, confidences = [], []

                for review in reviews:
                    try:
                        ratings.append(int(review.get("rating", "").split(":")[0]))
                        confidences.append(int(review.get("confidence", "").split(":")[0]))
                    except:
                        continue

                if ratings:
                    review_scores[pid] = {
                        "avg_rating": round(sum(ratings) / len(ratings), 2),
                        "avg_confidence": round(sum(confidences) / len(confidences), 2) if confidences else None,
                        "review_count": len(ratings)
                    }


In [8]:
def normalize_decision(raw_decision):
    """
    Normalize decision string into one of: 'Accept', 'Reject', or 'Other'.
    - Accept if it starts with 'Accept'
    - Reject if it starts with 'Reject'
    - Otherwise, Other
    """
    if not raw_decision:
        return "Other"

    lower_decision = raw_decision.strip().lower()
    if lower_decision.startswith("accept"):
        return "Accept"
    elif lower_decision.startswith("reject"):
        return "Reject"
    else:
        return "Other"

In [35]:
def estimate_weighted_decision_and_scores(query_text, k=10,modelInd=0):
    query_vec = models[modelInd].encode(query_text, convert_to_tensor=False)
    query_vec_np = np.array([query_vec], dtype='float32')
    D, I = indexes[modelInd].search(query_vec_np, k + 1)
    top_k_indices = I[0][1:]
    distances = D[0][1:]

    # Compute weights (softmax over negative distance)
    similarities = -distances
    weights = np.exp(similarities) / np.sum(np.exp(similarities))

    decision_weights = {}
    weighted_rating = 0
    weighted_conf = 0
    weight_total_rating = 0
    weight_total_conf = 0

    for i, idx in enumerate(top_k_indices):
        pid = paper_ids[idx]
        decision = normalize_decision(paper_decisions.get(pid, "Unknown"))
        weight = weights[i]

        # Weight decision
        decision_weights[decision] = decision_weights.get(decision, 0) + weight

        # Weight scores
        review = review_scores.get(pid, {})
        rating = review.get("avg_rating")
        confidence = review.get("avg_confidence")

        if rating is not None:
            weighted_rating += weight * rating
            weight_total_rating += weight

        if confidence is not None:
            weighted_conf += weight * confidence
            weight_total_conf += weight

    # Pick decision with highest total weight
    estimated_decision = max(decision_weights.items(), key=lambda x: x[1])[0]

    est_rating = round(weighted_rating / weight_total_rating, 2) if weight_total_rating else None
    est_conf = round(weighted_conf / weight_total_conf, 2) if weight_total_conf else None

    return {
        "estimated_decision": estimated_decision,
        "estimated_rating": est_rating,
        "estimated_confidence": est_conf,
        "decision_distribution": decision_weights
    }


In [34]:
from collections import Counter

def evaluate_accuracy_with_intro(n=100, k=10,modelInd=0):
    correct = 0
    total = 0
    confusion = Counter()

    for i in range(min(n, len(titles))):
        query_text = title_intro_texts[i]
        query_pid = paper_ids[i]
        true_decision = normalize_decision(paper_decisions.get(query_pid, "Unknown"))

        result = estimate_weighted_decision_and_scores(query_text, k,modelInd)
        pred_decision = result["estimated_decision"]

        if pred_decision == true_decision:
            correct += 1
        confusion[(true_decision, pred_decision)] += 1
        total += 1

    accuracy = correct / total if total > 0 else 0

    print(f"\n✅ Initial Results Accuracy (title+intro, top-{k}): {accuracy:.2f}")
    print("\n📊 Confusion Matrix (True → Predicted):")
    for (true, pred), count in confusion.items():
        print(f"{true:25s} → {pred:25s}: {count}")

    return accuracy, confusion


In [33]:
def evaluate_single(i, k=10,modelInd=0):
    query_text = title_intro_texts[i]
    query_pid = paper_ids[i]
    true_decision = paper_decisions.get(query_pid, "Unknown")
    query_title = titles[i]

    # Estimate using weighted method
    result = estimate_weighted_decision_and_scores(query_text, k)
    pred_decision = result["estimated_decision"]
    decision_weights = result["decision_distribution"]

    print(f"\n🔍 Original Paper [{query_pid}]:")
    print(f"Title: {query_title}")
    print(f"True Decision: {true_decision}")
    print(f"Predicted Decision: {pred_decision}")
    print("\n📋 Similar Papers and Their Decisions:")

    # Retrieve nearest neighbors and distances
    query_vec = models[modelInd].encode(query_text, convert_to_tensor=False)
    query_vec_np = np.array([query_vec], dtype='float32')
    D, I = indexes[modelInd].search(query_vec_np, k + 1)
    top_k_indices = I[0][1:]  # skip self
    distances = D[0][1:]

    similarities = -distances
    weights = np.exp(similarities) / np.sum(np.exp(similarities))

    for rank, (idx, weight) in enumerate(zip(top_k_indices, weights), start=1):
        pid = paper_ids[idx]
        title = titles[idx]
        decision = paper_decisions.get(pid, "Unknown")
        print(f"{rank}. {title} [{pid}]")
        print(f"   Decision: {decision} | Weight: {weight:.3f}")

    print("\n📊 Weighted Decision Totals:")
    for label, w in sorted(decision_weights.items(), key=lambda x: -x[1]):
        print(f"{label}: {w:.3f}")


In [26]:
evaluate_single(12,5)


🔍 Original Paper [ICLR_2017_111]:
Title: Trusting SVM for Piecewise Linear CNNs
True Decision: Accept (Poster)
Predicted Decision: Accept

📋 Similar Papers and Their Decisions:
1. A Unified View of Piecewise Linear Neural Network Verification [NIPS_2018_442]
   Decision: Accept | Weight: 0.229
2. Piecewise Linear Neural Networks verification: A comparative study [ICLR_2018_713]
   Decision: Reject | Weight: 0.228
3. Piecewise Strong Convexity of Neural Networks [NIPS_2019_1163]
   Decision: Accept | Weight: 0.187
4. SVCCA: Singular Vector Canonical Correlation Analysis for Deep Learning Dynamics and Interpretability [NIPS_2017_582]
   Decision: Accept | Weight: 0.178
5. Cautious Deep Learning [ICLR_2019_1140]
   Decision: Reject | Weight: 0.178

📊 Weighted Decision Totals:
Accept: 0.594
Reject: 0.406


In [ ]:
# evaluate_accuracy_with_intro(n=len(titles), k=10)
results = []
k_values = [3, 5, 7, 10, 15]
for i in range(3):
    for k in k_values:
        out = evaluate_accuracy_with_intro(n=len(titles), k=k,modelInd=i)
        results.append((i, k, out[0]))
        
sorted_results = sorted(results, key=lambda x: (-x[2], x[1]))
best = sorted_results[0]
print("\n✅ Best Configuration:")
print(f"Model: {best[0]}")
print(f"Top-k: {best[1]}")
print(f"Accuracy: {best[2]:.4f}")



✅ Initial Results Accuracy (title+intro, top-3): 0.55

📊 Confusion Matrix (True → Predicted):
Accept                    → Accept                   : 55
Accept                    → Reject                   : 43
Accept                    → Other                    : 2

✅ Initial Results Accuracy (title+intro, top-5): 0.57

📊 Confusion Matrix (True → Predicted):
Accept                    → Accept                   : 57
Accept                    → Reject                   : 43

✅ Initial Results Accuracy (title+intro, top-7): 0.56

📊 Confusion Matrix (True → Predicted):
Accept                    → Accept                   : 56
Accept                    → Reject                   : 44

✅ Initial Results Accuracy (title+intro, top-10): 0.58

📊 Confusion Matrix (True → Predicted):
Accept                    → Reject                   : 42
Accept                    → Accept                   : 58

✅ Initial Results Accuracy (title+intro, top-15): 0.58

📊 Confusion Matrix (True → Predicted):
Ac